O objetivo principal deste notebook é realizar uma regressão multipla para verificar o preço total em R$ de um imóvel.
Irei escrever baseado no que já conheço, nos notebooks que vejo aqui e ir melhorando com o tempo. Irei adicionar comentários
Irei adicionar comentários em portugues e ingles

The main objective of this notebook is to perform a multiple regression to check the total price in R $ of a property.
I will write based on what I already know, on the notebooks I see here and get better with time. I will add comments
I will add comments in Portuguese and English

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/brasilian-houses-to-rent/houses_to_rent_v2.csv')
# Any results you write to the current directory are saved as output.

## Análise Exploratória de Dados / Explodaroty Data Analisys - EDA 

In [ ]:
# Visão geral dos dados / General view od data
df.head(15)

In [ ]:
# Os valores únicos da coluna cidade / Unique values from city colum
df.city.unique()

In [ ]:
df.info()

> Os tipos de dados (Dtype) são diferentes, transformar depois.
> The data types (Dtype) are different, transform it later.

In [ ]:
# Cidades mais populares / Most popular cities
df.groupby(['city'])['city'].aggregate(lambda x: x.count()/ 10692).plot(kind='pie',autopct='%.2f',fontsize=13);


> Nota-se que a maioria dos dados concentra-se em São Paulo o que favorece a análise.
> Note that most data is concentrated in São Paulo, which favors the analysis. 

In [ ]:
# Valor médio das casas por cidade /  Average Total value by city
df.groupby(['city'])['total (R$)'].aggregate(lambda x: x.mean()).plot(kind='bar',color=['blue','orange','green','red','purple']\
    , label='Average R$', title='Average R$ in a city', fontsize=15);

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,5))

sns.countplot(x='city', hue='animal', data=df, palette=sns.color_palette(), ax=axes[0]);
sns.countplot(x='city', hue='furniture', data=df, palette='Set1', ax=axes[1]);

> A maioria das casas aceitam animais e não possuem móveis.
> Most houses accept animals and have no furniture.

In [ ]:
sns.pairplot(df, hue = "city", corner = True, height=2.5, kind = 'reg')

> Pode-se notar a relação entre as variáveis, que não se parece tão linear. Isto será mais evidente no final da análise.
> One can notice the relationship between the variables, which does not seem so linear. This will be most evident at the end of the analysis.

## Correlação / Correlation

In [ ]:
correlacao = df.corr()

In [ ]:
# Mapa de calor da correlaçao / Heatmap fo Correlation
plt.figure(figsize=(8, 7))
sns.heatmap(correlacao, cmap="Oranges", center=0, annot=True,vmin=-1, vmax=1,linewidth=1, linecolor='w', square=True,mask= np.triu(correlacao));

Devido a alta correlação entre fire insurance e rent amount (0.99), irei retirar a fire insurance para evitar multicolinearidade

In [ ]:

plt.scatter(x ='fire insurance (R$)', y = 'rent amount (R$)', data = df, s = 100, alpha = 0.3, edgecolor = 'white');
plt.title('fire insurance  rent amounttal', fontsize = 16);
plt.ylabel('fire insurance (R$)', fontsize = 12);
plt.xlabel('rent amount (R$)', fontsize = 12);

> Aqui pode-se ver a alta correlação linear entre as duas variáveis selecionadas;
> Here you can see the high linear correlation between the two selected variables.

## Tratamento de Dados / Data Engineering

In [ ]:
# Como alguns dados deveriam ser do tipo inteiro (int64), mas aparecem como objeto, irei verificar quais são e transformar em inteiros
# As some data should be of the integer type (int64), but appear as an object, I will check who they are and transform them into integers
for i in df.select_dtypes('object'):
    print(i,df[i].unique())

> o sinal '-' em "floor" 
> the sign '-' in "floor"

In [ ]:
# Vou considerar o valor ' - ' como indicando uma casa e não apartamento para facilitar as análises e transformar em Inteiro
# I will consider the value '-' as indicating a house and not an apartment to facilitate analysis and transform it into Integer

df['floor'] = df['floor'].apply(lambda x: -1 if x == '-' else x).astype(int)    

In [ ]:
df.info()

In [ ]:
# transformando dados categoricos em dummys
# turning categorical data into dummys
dummy_city = pd.get_dummies(df.city, prefix='city')
dummy_animal = pd.get_dummies(df.animal, prefix='animal',drop_first = True) 
dummy_furniture = pd.get_dummies(df.furniture, prefix='furniture',drop_first = True)

print(dummy_furniture.head())

In [ ]:
# concatenando dummys/ concatenate dummys
df = pd.concat([df,dummy_city], axis=1)
df = pd.concat([df,dummy_animal], axis=1)
df = pd.concat([df,dummy_furniture], axis=1)

df.head()

In [ ]:
df.info()

## Regression

In [ ]:
correlacao_02 = df.corr()

In [ ]:
# Nova correlação com os dados transformados
# New correlation with transformed data
plt.figure(figsize=(15, 12))
sns.heatmap(correlacao_02, cmap="coolwarm", center=0, annot=True,vmin=-1, vmax=1,linewidth=1, linecolor='w', square=True,mask= np.triu(correlacao_02));

> Não mudou muita coisa e algumas variáveis deram pequena correlação negativa;
> Not much has changed and some variables gave a small negative correlation.

In [ ]:
df.columns

In [ ]:
# Decidi usar apenas as casas de São Paulo pois usando todas as cidades obtive mais erros;
# I decided to use only the houses in São Paulo because using all the cities I got more errors;

df_Sao_Paulo = df.loc[df['city'] == 'São Paulo']

In [ ]:
# Valores / Values


# Variável Dependente / Dependent Variable 
X_var = df_Sao_Paulo[['hoa (R$)','rent amount (R$)','property tax (R$)']]

# Variável Independente / Independent Variable 
y_var = df_Sao_Paulo['total (R$)']


In [ ]:
# Dividindo em Treino e Teste
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_var, y_var, test_size = 0.2, random_state = 0)

In [ ]:
# Usando regressor
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
# Predizendo com r2 score / Predicting with r2 score
y_pred = regressor.predict(X_test)

from sklearn.metrics import r2_score
score=r2_score(y_test,y_pred)
score

In [ ]:
# Coeficientes 
regressor.coef_

In [ ]:
# Intercept
regressor.intercept_

### Testando com o primeiro valor do Data Frame

In [ ]:
valor_teste = df_Sao_Paulo.loc[0][12]
valor_teste

In [ ]:

df_Sao_Paulo.loc[0]

In [ ]:
#predizendo um valor, inserir:
'''  

X_var = df_Sao_Paulo[['hoa (R$)','rent amount (R$)','property tax (R$)']]

'''
predicted = regressor.predict([[2065,3300,211]])

print("O valor predito era pra ser {:03d} R$ e foi {:03d} R$.".format(valor_teste,int(predicted)))

diferenca = predicted - valor_teste

print("com uma diferença de {:02d} R$.".format(int(diferenca)))


> Pouca diferença...
> Little difference ...

## OLS Regression Results

O OLS permite uma visão mais detalhada da regressão permitindo uma melhor análise e ajuste

The OLS allows a more detailed view of the regression allowing for better analysis and adjustment

In [ ]:
import statsmodels.api as sm 
from termcolor import colored as cl # para mudar fonte do terminal e deixar em negrito

slr_model = sm.OLS(y_var, X_var) # Ordinary Least Squares 
slr_reg = slr_model.fit()

# sumario
print(cl(slr_reg.summary(),attrs = ['bold']))

* R-squared (uncentered): 1.000    
    * Deve estar proximo de 1
* F-statistic(significance of the regression): 1.773e+09
    * Deve estar proximo de 0
* AIC(It is calculated as number of parameters minus the likelihood of the overall model): 4.427e+04
    * Deve ser o menor possível
* Omnibus(normal distribuition of errors): 11494.474
    * Deve estar proximo de 1
* Durbin-Watson(homoscedasticity , constant variance of erros): 2.000
    * Deve estar entre 1 e 2
*  Jarque-Bera (JB)(distribution analysis of the regression errors): 70846183.405
    * Quanto maior, indica que os erros não estão distribuidos normalmente

**In English**

* R-squared (uncentered): 1,000
     * Must be close to 1
* F-statistic (significance of the regression): 1,773e + 09
     * Must be close to 0
* AIC (It is calculated as number of parameters minus the likelihood of the overall model): 4.427e + 04
     * Must be as small as possible
* Omnibus (normal distribution of errors): 11494.474
     * Must be close to 1
* Durbin-Watson (homoscedasticity, constant variance of errors): 2,000
     * Must be between 1 and 2
* Jarque-Bera (JB) (distribution analysis of the regression errors): 70846183.405
     * The larger, indicates that errors are not normally distributed

## Considerações finais /  Final considerations

O modelo ainda está muito impreciso, segundo os dados da OLS. Irei testar outros algoritmos como Àrvores de Decisão, Redes Neurais e outros. Qualquer ajuda é bem vinda.

The model is still very inaccurate, according to OLS data. I will test other algorithms like Decision Trees, Neural Networks and others. Every help is welcome.